## Imports

In [4]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [5]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 17th Sep. 24th


In [13]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [14]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [15]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [16]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [17]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-6-2024,Condo/Co-op,809 Republic Ct,Deerfield Beach,FL,33442.0,332500.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/809-...,Beaches MLS,RX-11006291,N,Y,26.296875,-80.158021


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [18]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [19]:
# Filter dates between. If only using data looking back one week, this shouldn't change the df
old_week = df[(df['SOLD DATE'] >= '2024-09-09') & (df['SOLD DATE'] <= '2024-09-15')]
df = df[(df['SOLD DATE'] >= '2024-09-16') & (df['SOLD DATE'] <= '2024-09-22')]

In [20]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
280,PAST SALE,2024-09-22,Condo/Co-op,3113 S Ocean Dr #302,Hallandale,FL,33009.0,300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/311...,Beaches MLS,F10440724,N,Y,25.977116,-80.121764
346,PAST SALE,2024-09-20,Condo/Co-op,3413 NW 44th St #104,Oakland Park,FL,33309.0,220000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/341...,MARMLS,A11604621,N,Y,26.180850,-80.194063
80,PAST SALE,2024-09-20,Condo/Co-op,1600 SW 127th Way Unit 303C,Pembroke Pines,FL,33027.0,200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1600-...,MARMLS,A11624417,N,Y,25.994249,-80.316365
211,PAST SALE,2024-09-20,Condo/Co-op,326 Harrison St Unit 205A,Hollywood,FL,33019.0,350000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/326-Harris...,MARMLS,A11616955,N,Y,26.010836,-80.116661
217,PAST SALE,2024-09-20,Condo/Co-op,8634 NW 12th St Unit C175,Fort Lauderdale,FL,33322.0,195000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/8634-NW-1...,Beaches MLS,F10453361,N,Y,26.138967,-80.265156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,PAST SALE,2024-09-16,Condo/Co-op,11205 W Atlantic Blvd #105,Coral Springs,FL,33071.0,229000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Springs/11205-...,Beaches MLS,F10453231,N,Y,26.241355,-80.274384
237,PAST SALE,2024-09-16,Condo/Co-op,2050 NE 39th St Unit 109 S,Lighthouse Point,FL,33064.0,163500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lighthouse-Point/205...,Beaches MLS,F10453193,N,Y,26.279105,-80.095465
110,PAST SALE,2024-09-16,Condo/Co-op,7500 NW 1st Ct #402,Plantation,FL,33317.0,252000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/7500-NW-1...,Beaches MLS,F10450581,N,Y,26.122734,-80.249218
233,PAST SALE,2024-09-16,Condo/Co-op,4142 NW 90th Ave #104,Coral Springs,FL,33065.0,190000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Springs/4142-N...,Beaches MLS,F10448465,N,Y,26.281201,-80.246812


In [21]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [22]:
df_top_ten = df.head(10)

## Collect Agent Information

In [23]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [24]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [25]:
response_df = pd.DataFrame(response_list)

In [26]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [27]:
df_top_ten = merged_df

## Current Week's Values

In [28]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

73


In [29]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$276,614


In [30]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$261


In [31]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$20,192,799


In [32]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [34]:
# comparison week total sales
print(f'Comparison week filtered sales: {(len(old_week))}')
print('-----------')
print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
print('-----------')
print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
print('-----------')
print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
print('-----------')

Comparison week filtered sales: 54
-----------
Comparison week average sales price: $257,589.35
-----------
Comparison week average psf: $260.44
-----------
Comparison week sales total: $13,909,825.00
-----------


In [35]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
54
Input Previous Week Condo Average Sales Price:
257589
Input Previous Week Condo Average PSF:
260
Input Previous Week Condo Sales total (ex: 198_000_000)
13_909_825


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [36]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [37]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [38]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [39]:
pd.set_option('display.max_columns',None)

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange


In [41]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange
1,PAST SALE,2024-09-16,Condo/Co-op,3100 NE 48th St #412,Fort Lauderdale,FL,33308.0,759000.0,2.0,2.0,Pilot House,1976.0,NaN,1977.0,NaN,384.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10450134,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Scott Cowles,Redfin Corporation,None,None,Lois Rutigliano,One Sotheby's Int'l Realty,None,None,2,blue
2,PAST SALE,2024-09-20,Condo/Co-op,2301 S Ocean Dr #1505,Hollywood,FL,33019.0,585000.0,2.0,2.0,CATANIA CONDO,1230.0,NaN,1978.0,NaN,476.0,1292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11653598,N,Y,25.997517,-80.116976,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,Svetlana Perez,"LP Global Realty, LLC",Ricardo Perez,"LP Global Realty, LLC",Daniela Lainville,Complete Realty Group,None,None,3,blue
3,PAST SALE,2024-09-17,Condo/Co-op,1500 S Ocean Blvd #1604,Lauderdale By The Sea,FL,33062.0,550000.0,2.0,2.0,Leisure Towers,1070.0,NaN,1970.0,NaN,514.0,975.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1500-S...,Beaches MLS,F10452525,N,Y,26.211826,-80.092066,https://www.redfin.com/FL/Pompano-Beach/1500-S...,John Sullivan,LoKation,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,4,blue
4,PAST SALE,2024-09-16,Condo/Co-op,511 SE 5th Ave #2214,Fort Lauderdale,FL,33301.0,550000.0,2.0,2.0,Nu River Landing,1379.0,NaN,2006.0,NaN,399.0,1156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Beaches MLS,RX-11005412,N,Y,26.116275,-80.139500,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Lamees Hassan,The Keyes Company,None,None,Zorka Dobreva,One Sotheby's Int'l Realty,None,None,5,blue
5,PAST SALE,2024-09-17,Condo/Co-op,3801 S Ocean Dr Unit 3V,Hollywood,FL,33019.0,500000.0,2.0,2.0,TIDES ON HOLLYWOOD BEACH,1176.0,NaN,1969.0,NaN,425.0,1068.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,MARMLS,A11624212,N,Y,25.989235,-80.118408,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,Noelia Quinonez,One Zero Realty,None,None,None,None,None,None,6,blue
6,PAST SALE,2024-09-18,Condo/Co-op,6263 Bay Club Dr #1,Fort Lauderdale,FL,33308.0,475000.0,2.0,2.0,Bay Colony Club,1200.0,NaN,1974.0,NaN,396.0,585.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Beaches MLS,F10448550,N,Y,26.208669,-80.102292,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Raphael Rucci,dba Raphael C. Rucci,None,None,Serafina Martell,Sera Martell Realty Inc,None,None,7,blue
7,PAST SALE,2024-09-19,Condo/Co-op,300 Three Islands Blvd #218,Hallandale Beach,FL,33009.0,467500.0,2.0,2.0,ANCHOR BAY CLUB CONDO,1692.0,NaN,1984.0,NaN,276.0,1158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/300...,MARMLS,A11650041,N,Y,25.990414,-80.125411,https://www.redfin.com/FL/Hallandale-Beach/300...,Sage Kantor,"FIP Realty Services, LLC",None,None,Sage Kantor,"FIP Realty Services, LLC",None,None,8,blue
8,PAST SALE,2024-09-19,Condo/Co-op,9152 Vineyard Lake Dr #9152,Plantation,FL,33324.0,455000.0,3.0,2.5,VINEYARDS OF PLANTATION C,1765.0,NaN,1987.0,NaN,258.0,510.0,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/9

In [44]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'Sept. 15 - Sept. 23')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [45]:
m.save('index.html')

## Data snagger

In [46]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [47]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [48]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_09232024


# CREATE TEMPLATE 

In [49]:
muni_set = set(df_top_ten['CITY'])

In [50]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [51]:
df_top_ten.reset_index(inplace=True,drop=True)

In [52]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [53]:
top_sale

'BEACH CLUB TWO CONDO at 1830 S Ocean Dr Unit LP4911 in Hallandale Beach'

In [54]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [55]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [56]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange,BEACH CLUB TWO CONDO at 1830 S Ocean Dr Unit L...
1,PAST SALE,2024-09-16,Condo/Co-op,3100 NE 48th St #412,Fort Lauderdale,FL,33308.0,759000.0,2.0,2.0,Pilot House,1976.0,NaN,1977.0,NaN,384.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10450134,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Scott Cowles,Redfin Corporation,None,None,Lois Rutigliano,One Sotheby's Int'l Realty,None,None,2,blue,Pilot House at 3100 NE 48th St #412 in Fort La...
2,PAST SALE,2024-09-20,Condo/Co-op,2301 S Ocean Dr #1505,Hollywood,FL,33019.0,585000.0,2.0,2.0,CATANIA CONDO,1230.0,NaN,1978.0,NaN,476.0,1292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11653598,N,Y,25.997517,-80.116976,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,Svetlana Perez,"LP Global Realty, LLC",Ricardo Perez,"LP Global Realty, LLC",Daniela Lainville,Complete Realty Group,None,None,3,blue,CATANIA CONDO at 2301 S Ocean Dr #1505 in Holl...
3,PAST SALE,2024-09-17,Condo/Co-op,1500 S Ocean Blvd #1604,Lauderdale By The Sea,FL,33062.0,550000.0,2.0,2.0,Leisure Towers,1070.0,NaN,1970.0,NaN,514.0,975.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1500-S...,Beaches MLS,F10452525,N,Y,26.211826,-80.092066,https://www.redfin.com/FL/Pompano-Beach/1500-S...,John Sullivan,LoKation,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,4,blue,Leisure Towers at 1500 S Ocean Blvd #1604 in L...
4,PAST SALE,2024-09-16,Condo/Co-op,511 SE 5th Ave #2214,Fort Lauderdale,FL,33301.0,550000.0,2.0,2.0,Nu River Landing,1379.0,NaN,2006.0,NaN,399.0,1156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Beaches MLS,RX-11005412,N,Y,26.116275,-80.139500,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Lamees Hassan,The Keyes Company,None,None,Zorka Dobreva,One Sotheby's Int'l Realty,None,None,5,blue,Nu River Landing at 511 SE 5th Ave #2214 in Fo...
5,PAST SALE,2024-09-17,Condo/Co-op,3801 S Ocean Dr Unit 3V,Hollywood,FL,33019.0,500000.0,2.0,2.0,TIDES ON HOLLYWOOD BEACH,1176.0,NaN,1969.0,NaN,425.0,1068.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,MARMLS,A11624212,N,Y,25.989235,-80.118408,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,Noelia Quinonez,One Zero Realty,None,None,None,None,None,None,6,blue,TIDES ON HOLLYWOOD BEACH at 3801 S Ocean Dr Un...
6,PAST SALE,2024-09-18,Condo/Co-op,6263 Bay Club Dr #1,Fort Lauderdale,FL,33308.0,475000.0,2.0,2.0,Bay Colony Club,1200.0,NaN,1974.0,NaN,396.0,585.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Beaches MLS,F10448550,N,Y,26.208669,-80.102292,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Raphael Rucci,dba Raphael C. Rucci,None,None,Serafina Martell,Sera Martell Realty Inc,None,None,7,blue,Bay Colony Club at 6263 Bay Club Dr #1 in Fort...
7,PAST SALE,2024-09-19,Condo/Co-op,300 Three Islands Blvd #218,Hallandale Beach,FL,33009.0,467500.0,2.0,2.0,ANCHOR BAY CLUB CONDO,1692.0,NaN,1984.0,NaN,276.0,1158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/300...,MARMLS,A11650041,N,Y,25.990414,-80.125411,https://

In [57]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: BEACH CLUB TWO CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $433,000M to $1,380,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Plantation, Hollywood, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 73 condo sales totaling $20,192,799 million from Sep. 17th to Sep. 24th. The previous week, brokers closed 54 condo sales totaling $13,909,825.

Last week’s units sold for an average of $276,614, higher than the $257,589 average price from the previous week. The average pri

In [58]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [59]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [60]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange,BEACH CLUB TWO CONDO at 1830 S Ocean Dr Unit L...
1,PAST SALE,2024-09-16,Condo/Co-op,3100 NE 48th St #412,Fort Lauderdale,FL,33308.0,759000.0,2.0,2.0,Pilot House,1976.0,NaN,1977.0,NaN,384.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10450134,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Scott Cowles,Redfin Corporation,None,None,Lois Rutigliano,One Sotheby's Int'l Realty,None,None,2,blue,Pilot House at 3100 NE 48th St #412 in Fort La...
2,PAST SALE,2024-09-20,Condo/Co-op,2301 S Ocean Dr #1505,Hollywood,FL,33019.0,585000.0,2.0,2.0,CATANIA CONDO,1230.0,NaN,1978.0,NaN,476.0,1292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11653598,N,Y,25.997517,-80.116976,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,Svetlana Perez,"LP Global Realty, LLC",Ricardo Perez,"LP Global Realty, LLC",Daniela Lainville,Complete Realty Group,None,None,3,blue,CATANIA CONDO at 2301 S Ocean Dr #1505 in Holl...
3,PAST SALE,2024-09-17,Condo/Co-op,1500 S Ocean Blvd #1604,Lauderdale By The Sea,FL,33062.0,550000.0,2.0,2.0,Leisure Towers,1070.0,NaN,1970.0,NaN,514.0,975.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1500-S...,Beaches MLS,F10452525,N,Y,26.211826,-80.092066,https://www.redfin.com/FL/Pompano-Beach/1500-S...,John Sullivan,LoKation,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,4,blue,Leisure Towers at 1500 S Ocean Blvd #1604 in L...
4,PAST SALE,2024-09-16,Condo/Co-op,511 SE 5th Ave #2214,Fort Lauderdale,FL,33301.0,550000.0,2.0,2.0,Nu River Landing,1379.0,NaN,2006.0,NaN,399.0,1156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Beaches MLS,RX-11005412,N,Y,26.116275,-80.139500,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Lamees Hassan,The Keyes Company,None,None,Zorka Dobreva,One Sotheby's Int'l Realty,None,None,5,blue,Nu River Landing at 511 SE 5th Ave #2214 in Fo...
5,PAST SALE,2024-09-17,Condo/Co-op,3801 S Ocean Dr Unit 3V,Hollywood,FL,33019.0,500000.0,2.0,2.0,TIDES ON HOLLYWOOD BEACH,1176.0,NaN,1969.0,NaN,425.0,1068.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,MARMLS,A11624212,N,Y,25.989235,-80.118408,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,Noelia Quinonez,One Zero Realty,None,None,None,None,None,None,6,blue,TIDES ON HOLLYWOOD BEACH at 3801 S Ocean Dr Un...
6,PAST SALE,2024-09-18,Condo/Co-op,6263 Bay Club Dr #1,Fort Lauderdale,FL,33308.0,475000.0,2.0,2.0,Bay Colony Club,1200.0,NaN,1974.0,NaN,396.0,585.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Beaches MLS,F10448550,N,Y,26.208669,-80.102292,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Raphael Rucci,dba Raphael C. Rucci,None,None,Serafina Martell,Sera Martell Realty Inc,None,None,7,blue,Bay Colony Club at 6263 Bay Club Dr #1 in Fort...
7,PAST SALE,2024-09-19,Condo/Co-op,300 Three Islands Blvd #218,Hallandale Beach,FL,33009.0,467500.0,2.0,2.0,ANCHOR BAY CLUB CONDO,1692.0,NaN,1984.0,NaN,276.0,1158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/300...,MARMLS,A11650041,N,Y,25.990414,-80.125411,https://

In [62]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: BEACH CLUB TWO CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $433,000M to $1,380,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Plantation, Hollywood, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 73 condo sales totaling $20,192,799 million from Sep. 17th to Sep. 24th. The previous week, brokers closed 54 condo sales totaling $13,909,825.

Last week’s units sold for an average of $276,614, higher than the $257,589 average price from the previous week. The average pri

In [63]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Hallandale-Beach/1830-S-Ocean-Dr-33009/unit-LP4911/home/104428088


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/3100-NE-48th-St-33308/unit-412/home/41707359


In [65]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/3031-N-Ocean-Blvd-33308/unit-1802/home/41715218


In [66]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Plantation/9152-Vineyard-Lake-Dr-33324/unit-9152/home/190927853


In [67]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Hallandale-Beach/1830-S-Ocean-Dr-33009/unit-LP4911/home/104428088


In [68]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Plantation/9152-Vineyard-Lake-Dr-33324/unit-9152/home/190927853


In [69]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-18,Condo/Co-op,1830 S Ocean Dr Unit LP4911,Hallandale Beach,FL,33009.0,1380000.0,3.0,2.0,BEACH CLUB TWO CONDO,1554.0,NaN,2006.0,NaN,888.0,1233.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/183...,MARMLS,A11621916,N,Y,25.984911,-80.118344,https://www.redfin.com/FL/Hallandale-Beach/183...,Juliana Tobon,Cervera Real Estate Inc,None,None,Sebastian Pappalettera,"Infinite Projects Realty, LLC.",None,None,1,orange,BEACH CLUB TWO CONDO at 1830 S Ocean Dr Unit L...
1,PAST SALE,2024-09-16,Condo/Co-op,3100 NE 48th St #412,Fort Lauderdale,FL,33308.0,759000.0,2.0,2.0,Pilot House,1976.0,NaN,1977.0,NaN,384.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10450134,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Scott Cowles,Redfin Corporation,None,None,Lois Rutigliano,One Sotheby's Int'l Realty,None,None,2,blue,Pilot House at 3100 NE 48th St #412 in Fort La...
2,PAST SALE,2024-09-20,Condo/Co-op,2301 S Ocean Dr #1505,Hollywood,FL,33019.0,585000.0,2.0,2.0,CATANIA CONDO,1230.0,NaN,1978.0,NaN,476.0,1292.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11653598,N,Y,25.997517,-80.116976,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,Svetlana Perez,"LP Global Realty, LLC",Ricardo Perez,"LP Global Realty, LLC",Daniela Lainville,Complete Realty Group,None,None,3,blue,CATANIA CONDO at 2301 S Ocean Dr #1505 in Holl...
3,PAST SALE,2024-09-17,Condo/Co-op,1500 S Ocean Blvd #1604,Lauderdale By The Sea,FL,33062.0,550000.0,2.0,2.0,Leisure Towers,1070.0,NaN,1970.0,NaN,514.0,975.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1500-S...,Beaches MLS,F10452525,N,Y,26.211826,-80.092066,https://www.redfin.com/FL/Pompano-Beach/1500-S...,John Sullivan,LoKation,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,4,blue,Leisure Towers at 1500 S Ocean Blvd #1604 in L...
4,PAST SALE,2024-09-16,Condo/Co-op,511 SE 5th Ave #2214,Fort Lauderdale,FL,33301.0,550000.0,2.0,2.0,Nu River Landing,1379.0,NaN,2006.0,NaN,399.0,1156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Beaches MLS,RX-11005412,N,Y,26.116275,-80.139500,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Lamees Hassan,The Keyes Company,None,None,Zorka Dobreva,One Sotheby's Int'l Realty,None,None,5,blue,Nu River Landing at 511 SE 5th Ave #2214 in Fo...
5,PAST SALE,2024-09-17,Condo/Co-op,3801 S Ocean Dr Unit 3V,Hollywood,FL,33019.0,500000.0,2.0,2.0,TIDES ON HOLLYWOOD BEACH,1176.0,NaN,1969.0,NaN,425.0,1068.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,MARMLS,A11624212,N,Y,25.989235,-80.118408,https://www.redfin.com/FL/Hollywood/3801-S-Oce...,Noelia Quinonez,One Zero Realty,None,None,None,None,None,None,6,blue,TIDES ON HOLLYWOOD BEACH at 3801 S Ocean Dr Un...
6,PAST SALE,2024-09-18,Condo/Co-op,6263 Bay Club Dr #1,Fort Lauderdale,FL,33308.0,475000.0,2.0,2.0,Bay Colony Club,1200.0,NaN,1974.0,NaN,396.0,585.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Beaches MLS,F10448550,N,Y,26.208669,-80.102292,https://www.redfin.com/FL/Fort-Lauderdale/6263...,Raphael Rucci,dba Raphael C. Rucci,None,None,Serafina Martell,Sera Martell Realty Inc,None,None,7,blue,Bay Colony Club at 6263 Bay Club Dr #1 in Fort...
7,PAST SALE,2024-09-19,Condo/Co-op,300 Three Islands Blvd #218,Hallandale Beach,FL,33009.0,467500.0,2.0,2.0,ANCHOR BAY CLUB CONDO,1692.0,NaN,1984.0,NaN,276.0,1158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/300...,MARMLS,A11650041,N,Y,25.990414,-80.125411,https://

## Time on Market Calculator

In [79]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 11) ## List (Earlier) date
date2 = datetime(2024, 9, 18) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

69


## Clean CSV for Datawrapper Chart

In [70]:
chart_df = df_top_ten

In [71]:
chart_df = chart_df.fillna(" ")

In [72]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [73]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [74]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [75]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")